# CSCE 636: Deep Learning
# UIN: 828002068
# Name: Mamoon Masud
# Submission - 8 
# Architecture - Transfer Learning ( Inception-v1 Inflated 3D ConvNet )
# Activity Recognition Project - Testing Evaluation Code

In [1]:
# Run if you haven't installed Keras Video Generators
!pip install keras-video-generators


ERROR: Error checking for conflicts.
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\pip\_vendor\pkg_resources\__init__.py", line 3012, in _dep_map
    return self.__dep_map
  File "C:\ProgramData\Anaconda3\lib\site-packages\pip\_vendor\pkg_resources\__init__.py", line 2806, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\pip\_vendor\pkg_resources\__init__.py", line 3003, in _parsed_pkg_info
    return self._pkg_info
  File "C:\ProgramData\Anaconda3\lib\site-packages\pip\_vendor\pkg_resources\__init__.py", line 2806, in __getattr__
    raise AttributeError(attr)
AttributeError: _pkg_info

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\pip\_inter

In [ ]:
import os
import glob
import keras
from keras_video import VideoFrameGenerator
import keras_video.utils
import tensorflow
from tensorflow.keras import backend
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.layers import (Conv2D, MaxPooling3D, Conv3D,MaxPooling2D)
from tensorflow.keras.layers import Dense, Flatten, Dropout, ZeroPadding3D
from tensorflow.keras.layers import LSTM
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.optimizers import Adam, RMSprop
from collections import deque
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import BatchNormalization
from keras.utils.vis_utils import plot_model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
import os.path
import time


# Loading the Model From the saved JSON File

In [12]:
# Set the path to the saved location of the JSON file. 
base_path = 'Enter Path where model and weight files are located'
model_path = os.path.join(base_path,'sub_8_model.json' )
with open(model_path, 'r') as json_file:
    json_savedModel= json_file.read()
#load the model architecture 
model_j = tensorflow.keras.models.model_from_json(json_savedModel)
model_j.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
i3d_inception (Model)        (None, 1, 1, 1, 1024)     12294544  
_________________________________________________________________
dropout_1 (Dropout)          (None, 1, 1, 1, 1024)     0         
_________________________________________________________________
Conv3d_6a_1x1 (Conv3D)       (None, 1, 1, 1, 200)      205000    
_________________________________________________________________
reshape_1 (Reshape)          (None, 1, 200)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                67840     
_________________________________________________________________
dense_1 (Dense)              (None, 128)               8320      
_________________________________________________________________
dense_2 (Dense)              (None, 64)               

In [ ]:
# Load the saved model weight
weight_path = os.path.join(base_path, 'sub_8_weights.hdf5')
model_j.load_weights(weight_path)

# Defining the video generator for the test set

In [27]:
# use sub directories names as classes
classes = ['Door', 'Not_Door']
classes.sort()
# some global params
SIZE = (224, 224)
CHANNELS = 3
NBFRAME = 15
BS = 5
# pattern to get videos and classes
glob_pattern='/content/drive/My Drive/DL Proj/Dataset/Test/{classname}/*.mp4'
# for data augmentation
data_aug = keras.preprocessing.image.ImageDataGenerator(
    zoom_range=.1,
    horizontal_flip=True,
    rotation_range=8,
    width_shift_range=.2,
    height_shift_range=.2)
# Create video frame generator
test = VideoFrameGenerator(
    classes=classes, 
    glob_pattern=glob_pattern,
    nb_frames=NBFRAME,
    split_val=0.00001,  # 33% of the videos will be used as validation set. 
    shuffle=True,
    batch_size=BS,
    target_shape=SIZE,
    nb_channel=CHANNELS,
    transformation=data_aug,
    use_frame_cache=False)

class Door, validation count: 0, train count: 22
class Not_Door, validation count: 0, train count: 15
Total data: 2 classes for 37 files for train


In [ ]:
optimizer = Adam(lr=1e-5, decay=1e-6)
model_j.compile(optimizer=optimizer , 
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [37]:
histos = model_j.evaluate_generator(test, steps = 10, verbose = 1 )

10/10 [==============================] - 7s 696ms/step - loss: 0.1962 - accuracy: 0.9000


In [44]:
model_j.evaluate(test, steps = 20,verbose = 1 )

20/20 [==============================] - 14s 721ms/step - loss: 0.2821 - accuracy: 0.9000


[0.2821061909198761, 0.8999999761581421]